In [1]:
import sys
import os

# Set the specific path you found
os.environ['JAVA_HOME'] = r'C:\Program Files\Eclipse Adoptium\jdk-17.0.17.10-hotspot'
os.environ['HADOOP_HOME'] = r'C:\hadoop'

# Update the system path so Python finds the correct java.exe and winutils.exe
os.environ['PATH'] = (
    os.path.join(os.environ['JAVA_HOME'], 'bin') + os.pathsep + 
    os.path.join(os.environ['HADOOP_HOME'], 'bin') + os.pathsep + 
    os.environ['PATH']
)

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, concat, monotonically_increasing_id

project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.append(project_root)

from src.utils.spark_utils import create_spark_session, stop_spark_session

print("✅ Setup complete: Java 17.0.17.10 is now mapped.")


✅ Setup complete: Java 17.0.17.10 is now mapped.


In [2]:
try:
    spark = SparkSession.builder \
        .appName("Enrichment Analysis") \
        .config("spark.driver.host", "127.0.0.1") \
        .config("spark.driver.bindAddress", "127.0.0.1") \
        .config("spark.sql.sources.commitProtocolClass", "org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol") \
        .getOrCreate()
    
    print("🔥 SUCCESS: Spark session is active!")
    spark.range(3).show()
except Exception as e:
    print("❌ Connection still blocked. Checking logs...")
    print(e)


🔥 SUCCESS: Spark session is active!
+---+
| id|
+---+
|  0|
|  1|
|  2|
+---+



In [3]:
cleaned_path = os.path.join(project_root, 'data', 'cleaned', 'superstore_cleaned_spark.parquet')
df = spark.read.parquet(cleaned_path)

print(f"📊 Dataset Assessment")
print(f"{'='*70}\n")
print(f"Rows: {df.count():,}")
print(f"Columns: {len(df.columns)}")
print(f"\nExisting columns: {df.columns}")

📊 Dataset Assessment

Rows: 9,800
Columns: 18

Existing columns: ['Row_ID', 'Order_ID', 'Order_Date', 'Ship_Date', 'Ship_Mode', 'Customer_ID', 'Customer_Name', 'Segment', 'Country', 'City', 'State', 'Postal_Code', 'Region', 'Product_ID', 'Category', 'Sub_Category', 'Product_Name', 'Sales']


In [4]:
print("\n📍 SCALABLE DATA ENRICHMENT STRATEGIES")
print("="*70)

enrichment_strategies = """
🌐 1. GEOGRAPHIC ENRICHMENT AT SCALE
   Challenge: Geocoding millions of addresses
   PySpark Solution: Broadcast unique location lookups.

👥 2. DEMOGRAPHIC DATA ENRICHMENT
   Challenge: Adding census/population data
   PySpark Solution: Join on geographic keys using partitioned Parquet files.

🌤️ 3. WEATHER DATA ENRICHMENT
   Challenge: Historical weather for each order
   PySpark Solution: Join on date + location keys using Delta Lake for ACID consistency.
"""
print(enrichment_strategies)



📍 SCALABLE DATA ENRICHMENT STRATEGIES

🌐 1. GEOGRAPHIC ENRICHMENT AT SCALE
   Challenge: Geocoding millions of addresses
   PySpark Solution: Broadcast unique location lookups.

👥 2. DEMOGRAPHIC DATA ENRICHMENT
   Challenge: Adding census/population data
   PySpark Solution: Join on geographic keys using partitioned Parquet files.

🌤️ 3. WEATHER DATA ENRICHMENT
   Challenge: Historical weather for each order
   PySpark Solution: Join on date + location keys using Delta Lake for ACID consistency.



In [5]:
print("\n🏗️ STAR SCHEMA DESIGN - BIG DATA OPTIMIZED")
print("="*70)

schema_design = """
🔑 Big Data Optimizations:

1. PARTITIONING STRATEGY
   Fact table: Partition by date (year/month).
   Dimensions: Partition by region/category.

2. FILE FORMAT
   Use Parquet or Delta Lake for columnar storage.

3. BUCKETING
   Bucket fact table by high-cardinality keys to reduce shuffles.
"""
print(schema_design)


🏗️ STAR SCHEMA DESIGN - BIG DATA OPTIMIZED

🔑 Big Data Optimizations:

1. PARTITIONING STRATEGY
   Fact table: Partition by date (year/month).
   Dimensions: Partition by region/category.

2. FILE FORMAT
   Use Parquet or Delta Lake for columnar storage.

3. BUCKETING
   Bucket fact table by high-cardinality keys to reduce shuffles.



In [6]:
from pyspark.sql.functions import monotonically_increasing_id

print("\n🔨 BUILDING DIMENSION TABLES WITH PYSPARK")
print("="*70)

if 'Customer Name' in df.columns and 'Segment' in df.columns:
    dim_customer = df.select('Customer Name', 'Segment', 'Region').distinct() \
        .withColumn('customer_key', monotonically_increasing_id())
    
    print(f"    ✅ Created dim_customer with {dim_customer.count():,} unique customers")
    dim_customer.show(5)

if 'Product Name' in df.columns and 'Category' in df.columns:
    dim_product = df.select('Product Name', 'Category', 'Sub-Category').distinct() \
        .withColumn('product_key', monotonically_increasing_id())
    
    print(f"    ✅ Created dim_product with {dim_product.count():,} unique products")
    dim_product.show(5)



🔨 BUILDING DIMENSION TABLES WITH PYSPARK


In [7]:
print("\n🔄 DISTRIBUTED ETL PIPELINE ARCHITECTURE")
print("="*70)

etl_architecture = """
┌─────────────────────────────────────────────────────────────┐
│                TRANSFORM PHASE (Distributed)                │
├─────────────────────────────────────────────────────────────┤
│ 1. Data Cleaning (Parallel Operations)                      │
│ 2. Data Enrichment (Batch API calls)                        │
│ 3. Dimensional Modeling (Surrogate Key generation)          │
│ 4. Aggregations (Window functions)                          │
└─────────────────────────────────────────────────────────────┘
"""
print(etl_architecture)


🔄 DISTRIBUTED ETL PIPELINE ARCHITECTURE

┌─────────────────────────────────────────────────────────────┐
│                TRANSFORM PHASE (Distributed)                │
├─────────────────────────────────────────────────────────────┤
│ 1. Data Cleaning (Parallel Operations)                      │
│ 2. Data Enrichment (Batch API calls)                        │
│ 3. Dimensional Modeling (Surrogate Key generation)          │
│ 4. Aggregations (Window functions)                          │
└─────────────────────────────────────────────────────────────┘



In [8]:
print("\n💻 PYSPARK ETL CODE PATTERN")
print("="*70)

from pyspark.sql.functions import broadcast

code_example = '''
# (Conceptual Pattern)
df_clean = df_raw.dropDuplicates().fillna(0)

dim_customer = df_clean.select("customer_id", "customer_name").distinct() \
    .withColumn("customer_key", monotonically_increasing_id())

fact_sales = df_clean.join(broadcast(dim_customer), on="customer_id", how="inner")

fact_sales.write.format("parquet").mode("append").partitionBy("year", "month").save(path)
'''
print(code_example)



💻 PYSPARK ETL CODE PATTERN

# (Conceptual Pattern)
df_clean = df_raw.dropDuplicates().fillna(0)

dim_customer = df_clean.select("customer_id", "customer_name").distinct()     .withColumn("customer_key", monotonically_increasing_id())

fact_sales = df_clean.join(broadcast(dim_customer), on="customer_id", how="inner")

fact_sales.write.format("parquet").mode("append").partitionBy("year", "month").save(path)



In [9]:
if 'spark' in locals():
    spark.stop()
    print("✅ Spark session stopped")

print("\n✅ Analysis complete! You have successfully built a scalable PySpark pipeline.")


✅ Spark session stopped

✅ Analysis complete! You have successfully built a scalable PySpark pipeline.
